In [27]:
%matplotlib inline

In [46]:
from pylab import *
import pylab
import matplotlib as mpl
import numpy
import lnm
from graph import Graph
import ipywidgets as widgets
from IPython.display import display, HTML
mpl.rcParams['savefig.dpi'] = 150
OPTIONS = {'FSM (HEAD)': 'results_fsm_head.txt',
           'KCFA (6.2.1)': 'results_kcfa_6_2_1.txt',
           'Snake (HEAD)': 'results_snake_head.txt',
           'Snake (6.2.1)': 'results_snake_6_2_1.txt',
           'Tetris (6.2.1)': 'results_tetris_6_2_1.txt',
           'Tetris (HEAD)': 'results_tetris_head.txt',
           'Sieve': 'results_sieve_6_2_1.txt',
           'Suffix Tree (6.2.1)': 'results_suffixtree_6_2_1.txt',
           'Synth (HEAD)':'results_synth_head.txt',
           'Synth (6.2.1)': 'results_synth_6_2_1.txt'}
s = widgets.Dropdown(options=OPTIONS)
all_data_files = ['results_fsm_head.txt',
                  'results_snake_head.txt',
                  'results_tetris_head.txt',
                  'results_sieve_6_2_1.txt',
                  'results_kcfa_6_2_1.txt',
                  'results_suffixtree_6_2_1.txt',
                  'results_synth_head.txt']

In [47]:
display(s)

In [44]:
HTML("<h1>Filename: '" + s.value + "'</h1>")

# Select Benchmark:

In [45]:
graph = lnm.read_data(s.value)
variations, data = graph.ungraph()
hidden_untyped = data[0,2]
pycket_untyped = data[0,1]
racket_untyped = data[0,0]
hidden = data[:,2]
pycket = data[:,1]
racket = data[:,0]

IOError: results_sieve.txt not found.

In [ ]:
# Runtime of Pycket as a function of Racket
ordered_data = np.copy(data)
ordered_data.sort(axis=0)
ohidden = ordered_data[:,2]
opycket = ordered_data[:,1]
oracket = ordered_data[:,0]
pylab.plot(oracket, ohidden, 'r', label='hidden')
pylab.plot(oracket, opycket, 'g', label='pycket')
pylab.plot(oracket, oracket, 'b', label='racket')
pylab.title('runtime as a function of Racket runtime')
pylab.xlabel('Racket runtime (ms)')
pylab.ylabel('system runtime (ms)')
pylab.legend(loc='best')

In [ ]:
pylab.plot(opycket, opycket, 'g', label='pycket')
pylab.plot(opycket, ohidden, 'r', label='hidden')
pylab.title('Runtime w.r.t Pycket')
pylab.xlabel('Pycket runtime')
pylab.ylabel('system runtime')
pylab.legend(loc='best')

In [ ]:
from scipy.stats import cumfreq
from pandas      import DataFrame
from ipy_table   import *

hidden_slowdown = hidden / float(hidden_untyped)
pycket_slowdown = pycket / float(pycket_untyped)
racket_slowdown = racket / float(racket_untyped)

def make_slowdown_data(fname):
    data = np.genfromtxt(fname, usecols=(1,2,3))
    hidden_untyped = float(data[0,2])
    pycket_untyped = float(data[0,1])
    racket_untyped = float(data[0,0])
    hidden = data[:,2]
    pycket = data[:,1]
    racket = data[:,0]
    weights = np.ones(len(hidden)) / float(len(hidden))
    return racket / racket_untyped, pycket / pycket_untyped, hidden / hidden_untyped, weights

def compute_deliverable(data):
    th = sum(data < 3)
    oh = sum(numpy.logical_and(data >= 3, data < 10))
    return [max(data), numpy.mean(data), np.median(data), "%d (%0.2f%%)" % (th, th / float(len(data)) * 100), "%d (%0.2f%%)" % (oh, oh / float(len(data)) * 100)]

def make_deliverable_table(**kwargs):
    lst = [["", "Max overhead", "Mean overhead", "Median overhead", "300-deliverable", "300/1000-usable"]]
    for name, data in kwargs.iteritems():
        lst.append([name] + compute_deliverable(data))
    tbl = make_table(map(list, zip(*lst)))
    apply_theme('basic_both')
    return tbl

def slowdown_cdf(*args, **kwargs):
    weights = kwargs.get('weights', None)
    entries = 0
    for data, color, label in args:
        if weights is not None:
            counts, bin_edges = numpy.histogram(data, weights=weights, bins=len(data))
            entries = np.sum(weights)
        else:
            counts, bin_edges = numpy.histogram(data, bins=len(data))
            entries = len(data)
        cdf = numpy.cumsum(counts)
        pylab.plot(bin_edges[:-1], cdf, color=color, label=label)
    pylab.axvline(3, color='y')
    pylab.axvline(10, color='k')
    pylab.axhline(len(data) * 0.6, color='c', ls='--')
    pylab.xlabel('slowdown factor')
    pylab.xlim((1,20))
    pylab.ylim((0,entries))
    pylab.ylabel('number below')
    pylab.legend(loc='best')
    
def compute_lnm_deliverable(slowdown_graph, L):
    graph = lnm.compute_lnm_times(slowdown_graph, L)
    data = graph.ungraph()[1]
    rs = data[:,0]
    ps = data[:,1]
    hs = data[:,2]
    return rs, ps, hs, make_deliverable_table(racket=rs, pycket=ps,hidden=hs)
    
rs, ps, hs, weights = zip(*[make_slowdown_data(fname) for fname in all_data_files])
rs = reduce(np.append, rs)
ps = reduce(np.append, ps)
hs = reduce(np.append, hs)
weights = reduce(np.append, weights)

# All Data

In [ ]:
slowdown_cdf((rs, 'b', 'racket'),
             (ps, 'g', 'pycket'),
             (hs, 'r', 'hidden'),
             weights=weights)

# Slowdown Summary Stats

In [ ]:
DataFrame({'racket':racket, 'pycket':pycket, 'hidden':hidden}).describe()

# Deliverability Stats

## L = 0

In [ ]:
slowdown_graph = lnm.fromkeyvals(variations, racket_slowdown, pycket_slowdown, hidden_slowdown)
racket_slowdown0, pycket_slowdown0, hidden_slowdown0, tbl = compute_lnm_deliverable(slowdown_graph, 0)
tbl

## L = 1

In [ ]:
racket_slowdown1, pycket_slowdown1, hidden_slowdown1, tbl = compute_lnm_deliverable(slowdown_graph, 1)
tbl

## L = 2

In [ ]:
racket_slowdown2, pycket_slowdown2, hidden_slowdown2, tbl = compute_lnm_deliverable(slowdown_graph, 2)
tbl

# Slowdown distribution

## L = 0

In [ ]:
slowdown_cdf((racket_slowdown0, 'b', 'racket'),
             (pycket_slowdown0, 'g', 'pycket'),
             (hidden_slowdown0, 'r', 'hidden'))

## L = 1

In [ ]:
slowdown_cdf((racket_slowdown1, 'b', 'racket'),
             (pycket_slowdown1, 'g', 'pycket'),
             (hidden_slowdown1, 'r', 'hidden'))

## L = 2

In [ ]:
slowdown_cdf((racket_slowdown2, 'b', 'racket'),
             (pycket_slowdown2, 'g', 'pycket'),
             (hidden_slowdown2, 'r', 'hidden'))